In [1]:
# !pip install geopy pandas folium


In [2]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import folium
import time

# Load CSV
df = pd.read_csv("Cities.csv")

# Initialize geocoder
geolocator = Nominatim(user_agent="sri_lanka_mapper")

# Function to handle timeouts
def geocode_city(city):
    try:
        location = geolocator.geocode(f"{city}, Sri Lanka", timeout=10)
        if location:
            return pd.Series([location.latitude, location.longitude])
        else:
            return pd.Series([None, None])
    except GeocoderTimedOut:
        time.sleep(1)
        return geocode_city(city)

# Geocode all cities
df[['Latitude', 'Longitude']] = df['City'].apply(geocode_city)

# Drop any rows where geocoding failed
df = df.dropna(subset=['Latitude', 'Longitude'])

# Create Folium map centered over Sri Lanka
sri_lanka_map = folium.Map(location=[7.8731, 80.7718], zoom_start=7)

# Add markers
for _, row in df.iterrows():
    folium.Marker(
        location=[row["Latitude"], row["Longitude"]],
        popup=row["City"],
        icon=folium.Icon(color="green", icon="map-marker")
    ).add_to(sri_lanka_map)

# Save to HTML
sri_lanka_map.save("sri_lanka_cities_map.html")
print("✅ Map saved as 'sri_lanka_cities_map.html'")


✅ Map saved as 'sri_lanka_cities_map.html'
